# Making a New Cancer Database

Taking the data from the federal government, specifically the [United States Cancer Statistics](https://www.cdc.gov/cancer/uscs/dataviz/download_data.htm). Looking specifically at the BYAREA.TXT file from the 1999-2017 zip file, which gives us the number of occurances of each cancer along with crude and age adjusted rates. This data set futher breaks down by sex and race while giving information of  location of cancer, and sex by year.

## Initial processing

Change their NaN/None ~/+ into np.nan, also want to cut down on file size by dropping the less interesting columns and decreasing the number of years.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('BYAREA.txt',sep="|",low_memory=False)
df.replace(['~','+'],np.nan)
#df.replace("+",np.nan)

,AREA,AGE_ADJUSTED_CI_LOWER,AGE_ADJUSTED_CI_UPPER,AGE_ADJUSTED_RATE,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,YEAR,CRUDE_CI_LOWER,CRUDE_CI_UPPER,CRUDE_RATE
0,Alabama,359.7,374.7,367.2,9299,Incidence,2293259,All Races,Female,All Cancer Sites Combined,1999,397.3,413.8,405.5
1,Alabama,160.6,170.5,165.5,4366,Mortality,2293259,All Races,Female,All Cancer Sites Combined,1999,184.8,196.1,190.4
2,Alabama,362.1,377.1,369.5,9474,Incidence,2302835,All Races,Female,All Cancer Sites Combined,2000,403.2,419.8,411.4
3,Alabama,160.8,170.6,165.7,4425,Mortality,2302835,All Races,Female,All Cancer Sites Combined,2000,186.5,197.9,192.2
4,Alabama,377.7,392.9,385.2,9971,Incidence,2309496,All Races,Female,All Cancer Sites Combined,2001,423.3,440.3,431.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145659,Wyoming,1.9,4.9,3.1,20,Mortality,551538,White,Male and Female,Urinary Bladder,2015,2.2,5.6,3.6
1145660,Wyoming,15.5,22.4,18.7,127,Incidence,549909,White,Male and Female,Urinary Bladder,2016,19.3,27.5,23.1
1145661,Wyoming,2.0,5.1,3.3,21,Mortality,549909,White,Male and Female,Urinary Bladder,2016,2.4,5.8,3.8
1145662,Wyoming,18.2,25.5,21.6,150,Incidence,544321,White,Male and Female,Urinary Bladder,2017,23.3,32.3,27.6


In [2]:
# Lets delete the less interesting data
df = df.drop(columns=["AGE_ADJUSTED_CI_LOWER","AGE_ADJUSTED_CI_UPPER","CRUDE_CI_LOWER","CRUDE_CI_UPPER"])

In [3]:
# Also, this file is so large, what happens if we restrict to five year intervals? (2019, 201)
# first lets delete any ranges by changing them to 0
df.YEAR= df.YEAR.apply(lambda x: np.where(x.isdigit(),x,0))
df.YEAR = df.YEAR.astype("int64")

In [4]:
# Lets look at what years we have the most data from
df.groupby('YEAR').count()

,AREA,AGE_ADJUSTED_RATE,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,CRUDE_RATE
YEAR,,,,,,,,,
0,57594,57594,57594,57594,57594,57594,57594,57594,57594
1999,55374,55374,55374,55374,55374,55374,55374,55374,55374
2000,55670,55670,55670,55670,55670,55670,55670,55670,55670
2001,56558,56558,56558,56558,56558,56558,56558,56558,56558
2002,56558,56558,56558,56558,56558,56558,56558,56558,56558
2003,57594,57594,57594,57594,57594,57594,57594,57594,57594
2004,57594,57594,57594,57594,57594,57594,57594,57594,57594
2005,57594,57594,57594,57594,57594,57594,57594,57594,57594
2006,57594,57594,57594,57594,57594,57594,57594,57594,57594


In [5]:
#have 57594 entries for each year, will delete all but 2017, 2012, 2007, 2002 to shrink the data set but give some yearly change
df_lessYears = df[(df['YEAR'] ==2002) | (df['YEAR'] ==2007) | (df['YEAR'] ==2012) | (df['YEAR'] ==2017)]

## Combining Mortality Data
Think to make this dataset more useful, the mortality events should be in the same rpw as the corresponding incident row.

To do so, will break apart the rows with mortalities and seperate the important data and recombine.

In [7]:
df_mortality = df_lessYears[df_lessYears['EVENT_TYPE']=='Mortality'] # seperate the mortality rows
df_removeMortRows = df_lessYears.drop(df_lessYears[df_lessYears['EVENT_TYPE']=='Mortality'].index) # drop the event column as no longer needed

,AREA,AGE_ADJUSTED_RATE,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,YEAR,CRUDE_RATE
6,Alabama,389.9,10151,Incidence,2314370,All Races,Female,All Cancer Sites Combined,2002,438.6
16,Alabama,396.1,11012,Incidence,2407275,All Races,Female,All Cancer Sites Combined,2007,457.4
26,Alabama,397.9,11921,Incidence,2479787,All Races,Female,All Cancer Sites Combined,2012,480.7
38,Alabama,397.4,12661,Incidence,2516043,All Races,Female,All Cancer Sites Combined,2017,503.2
46,Alabama,5.6,139,Incidence,2314370,All Races,Female,Brain and Other Nervous System,2002,6.0
...,...,...,...,...,...,...,...,...,...,...
1145622,Wyoming,15.6,89,Incidence,544321,White,Male and Female,Thyroid,2017,16.4
1145630,Wyoming,26.6,129,Incidence,477938,White,Male and Female,Urinary Bladder,2002,27.0
1145640,Wyoming,22.9,122,Incidence,508757,White,Male and Female,Urinary Bladder,2007,24.0
1145650,Wyoming,25.2,153,Incidence,543563,White,Male and Female,Urinary Bladder,2012,28.1


In [8]:
df_removeMortRows = df_removeMortRows.drop(columns=['EVENT_TYPE']) #Drop event column from indicendt data too
# rename columns for clarity
df_removeMortRows.columns = ["Area","Incidence.Age_Adjust_Rate","Incidence.Count","Population","Race","Sex","Site","Year","Incidence.Crude_Rate"] 

In [9]:
df_mortality.index = df_mortality.index -1 # fix indicies to line up, assumes mortality always on row below!

In [11]:
# Drop redundant columns in the mortality dataset and rename remaining
df_uniqueMortality = df_mortality.drop(columns=["AREA","POPULATION","RACE","SEX","YEAR","SITE","EVENT_TYPE"])
df_uniqueMortality.columns = ["Mortality.Age_Adjust_Rate","Mortality.Count","Mortality.Crude_Rate"]

,Mortality.Age_Adjust_Rate,Mortality.Count,Mortality.Crude_Rate
6,163.1,4407,190.4
16,156.4,4492,186.6
26,147.5,4568,184.2
38,138.5,4689,186.4
46,3.8,100,4.3
...,...,...,...
1145622,0.1,~,0.2
1145630,3.5,16,3.3
1145640,3.8,20,3.9
1145650,4.9,28,5.2


In [12]:
# recombine with a concatination
df_recombine =  pd.concat([df_removeMortRows,df_uniqueMortality.reindex(df_removeMortRows.index)],axis=1)

,Area,Incidence.Age_Adjust_Rate,Incidence.Count,Population,Race,Sex,Site,Year,Incidence.Crude_Rate,Mortality.Age_Adjust_Rate,Mortality.Count,Mortality.Crude_Rate
6,Alabama,389.9,10151,2314370,All Races,Female,All Cancer Sites Combined,2002,438.6,163.1,4407,190.4
16,Alabama,396.1,11012,2407275,All Races,Female,All Cancer Sites Combined,2007,457.4,156.4,4492,186.6
26,Alabama,397.9,11921,2479787,All Races,Female,All Cancer Sites Combined,2012,480.7,147.5,4568,184.2
38,Alabama,397.4,12661,2516043,All Races,Female,All Cancer Sites Combined,2017,503.2,138.5,4689,186.4
46,Alabama,5.6,139,2314370,All Races,Female,Brain and Other Nervous System,2002,6.0,3.8,100,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1145622,Wyoming,15.6,89,544321,White,Male and Female,Thyroid,2017,16.4,0.1,~,0.2
1145630,Wyoming,26.6,129,477938,White,Male and Female,Urinary Bladder,2002,27.0,3.5,16,3.3
1145640,Wyoming,22.9,122,508757,White,Male and Female,Urinary Bladder,2007,24.0,3.8,20,3.9
1145650,Wyoming,25.2,153,543563,White,Male and Female,Urinary Bladder,2012,28.1,4.9,28,5.2


## Final Processing

I am relatively happy with this data. Might be less helpful as it will require sorting by the user as I did not transpose each cancer/race/sex to its own column as I am not aware of an efficient way to do that. Instead boolean sorting can be done to get the data wanted by the each student.

Think I will make the meta data file by hand

Therefore can export to csv as final product (at least for this iteration):

In [35]:
df_recombine.to_csv('cancer_data.csv',header=False,index=False)

In [ ]:
# Making the meta data file
